In [8]:
import os

In [9]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [10]:
import redis

In [11]:
r = redis.Redis(host='127.0.0.1',port=6379,db=0, password=None, socket_timeout=None) #__init__

In [12]:
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})

True

In [13]:
r.get("Bahamas")

b'Nassau'

In [25]:
### datetime

In [14]:
import datetime

In [15]:
today = datetime.date.today()

In [16]:
visitors = {"dan", "jon", "alex"}

In [19]:
# r.sadd(today, *visitors) ## redis.exceptions.DataError: Invalid input of type: 'date'. Convert to a byte, string or number first.

In [20]:
stoday = today.isoformat() 

In [21]:
stoday

'2023-02-01'

In [22]:
r.sadd(stoday, *visitors)  # sadd: set-add

3

In [23]:
 r.smembers(stoday)

{b'alex', b'dan', b'jon'}

In [24]:
r.scard(today.isoformat())

3

In [26]:
### Example: PyHats.com

In [27]:
import random

In [28]:
random.seed(444)

In [29]:
hats = {f"hat:{random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

In [30]:
hats

{'hat:1326692461': {'color': 'black',
  'price': 49.99,
  'style': 'fitted',
  'quantity': 1000,
  'npurchased': 0},
 'hat:1236154736': {'color': 'maroon',
  'price': 59.99,
  'style': 'hipster',
  'quantity': 500,
  'npurchased': 0},
 'hat:56854717': {'color': 'green',
  'price': 99.99,
  'style': 'baseball',
  'quantity': 200,
  'npurchased': 0}}

In [136]:
import json

In [137]:
r1 = redis.Redis(db=1)

In [138]:
with r1.pipeline() as pipe:
    for h_id, hat in hats.items():
        print('h_id : ',h_id)
        print('hat : ',hat)
        json_test_dict = json.dumps(hat, ensure_ascii=False).encode('utf-8')
        r1.set(h_id, json_test_dict)
    pipe.execute()

h_id :  hat:1326692461
hat :  {'color': 'black', 'price': 49.99, 'style': 'fitted', 'quantity': 1000, 'npurchased': 0}
h_id :  hat:1236154736
hat :  {'color': 'maroon', 'price': 59.99, 'style': 'hipster', 'quantity': 500, 'npurchased': 0}
h_id :  hat:56854717
hat :  {'color': 'green', 'price': 99.99, 'style': 'baseball', 'quantity': 200, 'npurchased': 0}


In [139]:
json_test_dict = r1.get('hat:1326692461').decode('utf-8')
test_dict2 = dict(json.loads(json_test_dict))

In [140]:
test_dict2

{'color': 'black', 'price': 49.99, 'style': 'fitted', 'quantity': 1000, 'npurchased': 0}

In [141]:
r1.keys()

[b'hat:1236154736', b'hat:1326692461', b'hat:56854717']

In [153]:
from redis import StrictRedis
r2 = StrictRedis(db=2)
with r2.pipeline() as pipe:
    for h_id, hat in hats.items():
        print('h_id : ',h_id)
        print('hat : ',hat)
        r2.hset(h_id, mapping=hat)
    pipe.execute()

h_id :  hat:1326692461
hat :  {'color': 'black', 'price': 49.99, 'style': 'fitted', 'quantity': 1000, 'npurchased': 0}
h_id :  hat:1236154736
hat :  {'color': 'maroon', 'price': 59.99, 'style': 'hipster', 'quantity': 500, 'npurchased': 0}
h_id :  hat:56854717
hat :  {'color': 'green', 'price': 99.99, 'style': 'baseball', 'quantity': 200, 'npurchased': 0}


In [154]:
r2.bgsave()

True

In [156]:
r2.keys()

[b'hat:1236154736', b'hat:1326692461', b'hat:56854717']

In [157]:
r2.hincrby("hat:56854717", "quantity", -1)

199

In [158]:
r2.hget("hat:56854717", "quantity")

b'199'

In [159]:
r2.hincrby("hat:56854717", "npurchased", 1)

1